<a href="https://colab.research.google.com/github/coderANYA/AI-Course/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**fashion mnist dataset**

In [ ]:
import tensorflow as  tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fmnist.load_data()

In [ ]:
def show_image():
  from random import randint
  index = randint(0, 60000)
  plt.figure(figsize=(1,1))
  plt.imshow(train_images[index], cmap='gray')
  plt.title(f'label:{train_labels[index]}')
  plt.grid(False)
  plt.axis('off')
  plt.show()
show_image()

In [ ]:
# normalize the images (0 - 255) to (0 - 1)
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images.shape

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28,28, 1)
test_images = test_images.reshape(test_images.shape[0],28,28,1)

In [ ]:
# cnn model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape =  train_images.shape[1:]),  # 1 = Black & White image
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')    # output layer
])
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
model.summary()

Now train/fit the dataset

In [ ]:
epochs = 1
batch_size = 32
val_size = .2
patience = 5
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience)
board = tf.keras.callbacks.TensorBoard(log_dir = 'logs', histogram_freq=1)
history = model.fit(
    train_images, train_labels,
    epochs = epochs,
    batch_size = batch_size,
    validation_split = val_size,
    callbacks = [es, board],
    verbose = 1,
)

In [ ]:
# evaluate the model
model.evaluate(test_images, test_labels)

In [ ]:
from PIL import Image
def predict_img(img_path):
    class_names = ['T-shirt/top',
                'Trouser', 'Pullover',
                'Dress', 'Coat',
                'Sandal', 'Shirt',
                'Sneaker', 'Bag',
                'Ankle boot']
    img = Image.open(img_path).convert('L')             # load and make it grayscale
    img = img.resize((28, 28))                          # resize to 28x28
    img = np.array(img)                                 # convert to numpy array
    img = img / 255.0                                   # normalize
    img = img.reshape(1, 28, 28, 1)                     # reshape to (1, 28, 28, 1)
    prediction = model.predict(img)                     # predict
    name = class_names[np.argmax(prediction)]           # get the class name
    return {
        'prediction': np.argmax(prediction),
        'confidence': np.max(prediction),
        'name': name
    }